## Q2 -

#### Task A

We will be running Breadth-First Search algorithm to calculate the areas we need to reach to complete the requirements for each of the 2 cases, for each of the two citites mentioned. 

1 gallon of liquid fuel = 12194 Btu/gal

1 barrel = 42 gallons

1 Btu = 1055.06 J

12194 Btu/gal = 12.865 MJ/gallon

Biomass LHV = 18.6 MJ/kg dry basis

Efficiency = 50%

Effective biomass energy output = 9.3 MJ/kg dry basis








## Setup

We first setup the BFS as required. The adjacency matrix given is loaded as it will be used to locate neighbours of the counties currently in use. The steps to solve this problem are - 

1. Load the adjacency list. It contains neighbouring information for each county across the US. The value of a matrix cell is 1 if the counties on the correspponding row and column of the matrix are adjacent neighbours and 0 otherwise.
2. For this part, we are not interested in calcaulting the distance between counties, and hence we use BFS. The algorithm will start at the source and at each level, will keep adding the neighbours to its queue. Using this, we are ensured of exploiting the adjacent counties before moving ahead with an indirect neighbour (neighbour of neighbour).
3. The BFS execution stops when the target production value has been reached. After which, it prints all the counties used, as well as the land area used.
4. To calculate the target production, we need to make use of the data files provided. The data files provided are obtained for various biomass sources and there land area as well as production value (dry tonne) is summed up for an overall analysis.

In [1]:
import pandas as pd
import numpy as np
from collections import deque, defaultdict

In [2]:
def load_and_merge_biomass(files):
    dfs = []
    for f in files:
        df = pd.read_csv(f, dtype={"Fips Code": str})
        dfs.append(df[["Fips Code", "County", "Sqmi", "Production"]])
    merged = pd.concat(dfs)
    # aggregate by FIPS
    grouped = merged.groupby(["Fips Code", "County"], as_index=False).agg({
        "Production": "sum",
        "Sqmi": "sum"
    })

    # Convert tonnes/year → kg/day
    grouped["Production"] = grouped["Production"] * (1000 / 365)

    return grouped

In [3]:
def load_adjacency(file_path):
    """
    Loads adjacency matrix CSV (counties x counties) and converts to adjacency list
    """
    mat = pd.read_csv(file_path, index_col=0, dtype=str)
    # Ensure adjacency matrix FIPS are 5-character strings
    mat.index = mat.index.astype(str).str.zfill(5)
    mat.columns = mat.columns.astype(str).str.zfill(5)
    counties = mat.index.tolist()
    adj_list = defaultdict(list)
    for i, c1 in enumerate(counties):
        for j, c2 in enumerate(counties):
            if mat.iloc[i, j] == "1" and c1 != c2:  # adjacency
                adj_list[c1].append(c2)
    return adj_list

In [4]:
def bfs_collect(adj_list, biomass_df, start_fips, target_production):
    visited = set()
    q = deque([start_fips])
    
    total_prod = 0
    total_area = 0
    picked = []
    
    # Ensure Fips Code is string with 5 digits (leading zeros preserved)
    biomass_df["Fips Code"] = biomass_df["Fips Code"].astype(str).str.zfill(5)
    # map FIPS -> biomass info
    biomass_map = biomass_df.set_index("Fips Code").to_dict(orient="index")
    

    
    while q and total_prod < target_production:
        fips = q.popleft()
        if fips in visited:
            continue
        visited.add(fips)
        
        if fips in biomass_map:
            prod = biomass_map[fips]["Production"]
            area = biomass_map[fips]["Sqmi"]
            # Check if adding this county exceeds target
            if total_prod + prod > target_production:
                needed = target_production - total_prod
                frac = needed / prod  # fraction of county used
                total_prod += needed
                total_area += area * frac
                picked.append((fips, biomass_map[fips]["County"], needed, area * frac))
                break  # stop once target reached
            else:
                total_prod += prod
                total_area += area
                picked.append((fips, biomass_map[fips]["County"], prod, area))
        
        for neigh in adj_list[fips]:
            if neigh not in visited:
                q.append(neigh)
    
    return total_prod, total_area, picked

In [5]:
# Biomass files
files = [
        "billionton_23_forestry_mm-med.csv",
        "billionton_23_agri_mm-med.csv",
        "billionton_23_wastes_mm-med.csv"
    ]
biomass_df = load_and_merge_biomass(files)
    
# Load adjacency (from .csv adjacency matrix)
adj_list = load_adjacency("county_adjacency_matrix.csv")


(a)	Requirement = 150,000 barrels/day – 

150000 * 42 gallons/day = 6300000 gallons/day

Since 1 gallon of liquid fuel = 12.865 MJ/gallon

We require = 12194*63*(10^5) Btu/day

= 12194 * 63*(10^5)*(1055.06) J/day

= 81,052,030.332 MJ/day


Since effective biomass energy output = 9.3 MJ/kg dry basis. We need 81052030.332/9.3 = 8,715,272.078 kg (dry basis) each day !

In [6]:
requirement = 150000 # barrels/day
gallons_per_barrel = 42
lhv = 12194 # Btu/gallon
gallons_per_day = requirement * gallons_per_barrel
energy_required_per_day = gallons_per_day * lhv  # Btu/day
energy_joules_per_day = energy_required_per_day * 1055.06  # J/day
energy_mj_per_day = energy_joules_per_day / 1e6  ##MJ/day

print("Energy required per day (MJ):", energy_mj_per_day) #MJ/day

biomass_lhv = 18.6 # MJ/kg dry basis
efficiency = 0.50 # 50% efficiency

effective_biomass_energy_output = biomass_lhv * efficiency # MJ/kg dry basis
biomass_required_per_day = energy_mj_per_day / effective_biomass_energy_output # kg dry basis/day
print("Biomass required per day (kg dry basis):", biomass_required_per_day)

target_production = biomass_required_per_day # kg dry basis/day


Energy required per day (MJ): 81052030.332
Biomass required per day (kg dry basis): 8715272.078709677


### KANSAS CITY

In [7]:
start_fips = "29095" # Kansas City, MO


total_prod, total_area, picked = bfs_collect(adj_list, biomass_df, start_fips, target_production)
    
print("Total Production gathered:", total_prod) 
print("Total Area gathered (sqmi):", total_area)
print("Counties picked in BFS order:")
for f, name, prod, area in picked:
    print(f"{f} - {name} | Prod={prod:.2f} | Area={area:.2f}")

Total Production gathered: 8715272.078709677
Total Area gathered (sqmi): 74669.89933262936
Counties picked in BFS order:
29095 - Jackson County | Prod=1785990.52 | Area=14792.16
20091 - Johnson County | Prod=3658942.47 | Area=13917.68
20209 - Wyandotte County | Prod=263074.79 | Area=3123.80
29037 - Cass County | Prod=983183.64 | Area=16154.97
29047 - Clay County | Prod=797081.10 | Area=7764.73
29101 - Johnson County | Prod=1226999.56 | Area=18916.56


### PITTSBURGH

In [8]:
start_fips = "42003" # Pittsburgh, PA


total_prod_2, total_area_2, picked_2 = bfs_collect(adj_list, biomass_df, start_fips, target_production)
    
print("Total Production gathered:", total_prod_2)
print("Total Area gathered (sqmi):", total_area_2)
print("Counties picked in BFS order:")
for f, name, prod, area in picked_2:
    print(f"{f} - {name} | Prod={prod:.2f} | Area={area:.2f}")

Total Production gathered: 8715272.078709677
Total Area gathered (sqmi): 143294.65520821328
Counties picked in BFS order:
42003 - Allegheny County | Prod=2119902.52 | Area=20095.02
42007 - Beaver County | Prod=256391.56 | Area=10656.72
42019 - Butler County | Prod=1492132.03 | Area=21482.82
42125 - Washington County | Prod=1729322.36 | Area=23242.95
42129 - Westmoreland County | Prod=1759384.41 | Area=31077.90
39029 - Columbiana County | Prod=587141.75 | Area=11229.12
42073 - Lawrence County | Prod=278693.73 | Area=8320.94
54029 - Hancock County | Prod=23591.51 | Area=1231.58
42005 - Armstrong County | Prod=468712.22 | Area=15957.61


(b)	Requirement = 10,000 barrels/day – 

10000 * 42 gallons/day = 420000 gallons/day

Since 1 gallon of liquid fuel = 12.865 MJ/gallon

We require = 12194*42*(10^4) Btu/day

= 12194 * 42*(10^4)*(1055.06) J/day

= 5,403,468.6888 MJ/day

Since effective biomass energy output = 9.3 MJ/kg dry basis. We need 5403468.688/9.3 = 581,018.13 kg (dry basis) each day !



In [9]:
requirement = 10000 # barrels/day
gallons_per_barrel = 42
lhv = 12194 # Btu/gallon
gallons_per_day = requirement * gallons_per_barrel
energy_required_per_day = gallons_per_day * lhv  # Btu/day
energy_joules_per_day = energy_required_per_day * 1055.06  # J/day
energy_mj_per_day = energy_joules_per_day / 1e6  ##MJ/day

print("Energy required per day (MJ):", energy_mj_per_day) #MJ/day

biomass_lhv = 18.6 # MJ/kg dry basis
efficiency = 0.50 # 50% efficiency

effective_biomass_energy_output = biomass_lhv * efficiency # MJ/kg dry basis
biomass_required_per_day = energy_mj_per_day / effective_biomass_energy_output # kg dry basis/day
print("Biomass required per day (kg dry basis):", biomass_required_per_day)

target_production_2 = biomass_required_per_day # kg dry basis/day

Energy required per day (MJ): 5403468.6888
Biomass required per day (kg dry basis): 581018.1385806451


### KANSAS CITY

In [10]:
start_fips = "29095" # Kansas City, MO



total_prod_3, total_area_3, picked_3 = bfs_collect(adj_list, biomass_df, start_fips, target_production_2)
    
print("Total Production gathered:", total_prod_3)
print("Total Area gathered (sqmi):", total_area_3)
print("Counties picked in BFS order:")
for f, name, prod, area in picked_3:
    print(f"{f} - {name} | Prod={prod:.2f} | Area={area:.2f}")

Total Production gathered: 581018.1385806451
Total Area gathered (sqmi): 4812.18302667713
Counties picked in BFS order:
29095 - Jackson County | Prod=581018.14 | Area=4812.18


### PITTSBURGH

In [11]:
start_fips = "42003" # Pittsburgh, PA



total_prod_4, total_area_4, picked_4 = bfs_collect(adj_list, biomass_df, start_fips, target_production_2)
    
print("Total Production gathered:", total_prod_4)
print("Total Area gathered (sqmi):", total_area_4)
print("Counties picked in BFS order:")
for f, name, prod, area in picked_4:
    print(f"{f} - {name} | Prod={prod:.2f} | Area={area:.2f}")

Total Production gathered: 581018.1385806451
Total Area gathered (sqmi): 5507.598109805056
Counties picked in BFS order:
42003 - Allegheny County | Prod=581018.14 | Area=5507.60


### Task B

We have the data for dry tonne produced at each of the counties explored in BFS from the previous task. 

Now, fuel efficiency = 1.5 MJ per tonne-km

Original moisture = 50% weight, which means that we need to double the dry tonne weight of production from each county for transport computation in part (b). 

Here, an 8% decrease means a total water content of 42% by weight, this means that we need to multiply the dry tonne weight of production from each county by a factor of (100/58). 

We will then calculate the sum of distance from the centroid of each county to Kanas/Pittsburgh, and calculate the total energy consumption in each case. 

For the purpose of this exercise, we have obtained the centroids of each county required from the US Census bureau website. The data can be scraped, but we have coded it manually.


In [12]:
truck_fuel_efficiency = 1.5 # MJ/tonne-km

### KANSAS

Case 1 - 150,000 Barrels/day 

29095 - Jackson County, 
20091 - Johnson County,
20209 - Wyandotte County, 
29037 - Cass County, 
29047 - Clay County, 
29101 - Johnson County 

Case 2 - 10,000 Barrels/day 

29095 - Jackson County 



In [13]:
from haversine import haversine

# Reference county: Jackson County, MO (Kansas City)
kc = (39.007, -94.347)   # (lat, lon)

# Case 1 counties (150k bbl/day)
case1 = {
    "29095 - Jackson County, MO": (39.007, -94.347),
    "20091 - Johnson County, KS": (38.885, -94.823),
    "20209 - Wyandotte County, KS": (39.116, -94.764),
    "29037 - Cass County, MO": (38.642, -94.354),
    "29047 - Clay County, MO": (39.343, -94.425),
    "29101 - Johnson County, MO": (38.763, -93.800),
}


def compute(case_name, counties):
    print(f"\n=== {case_name} ===")
    total = 0.0
    for name, coords in counties.items():
        d = haversine(kc, coords)  # defaults to km
        total += d
        print(f"{name}: {d:.2f} km")
    print(f"--> Sum of distances: {total:.2f} km")
    return total

total = compute("Case 1 (150k bbl/day)", case1)




=== Case 1 (150k bbl/day) ===
29095 - Jackson County, MO: 0.00 km
20091 - Johnson County, KS: 43.34 km
20209 - Wyandotte County, KS: 37.99 km
29037 - Cass County, MO: 40.59 km
29047 - Clay County, MO: 37.96 km
29101 - Johnson County, MO: 54.57 km
--> Sum of distances: 214.45 km


Now, total production (dry kg/day), as calculated in Task A -
 
Total Production gathered: 8715272.078709677 kg ~ 8715.272 tonnes

In [14]:
total_distance = total*1.3 #kms
total_production = 8715.272 #tonnes/day (calculated in task A for Kansas City)
pelletized_weight = total_production * (100/58) #kg/day

total_energy = total_distance * pelletized_weight * truck_fuel_efficiency #MJ/day
print(f"Total energy for trucking (MJ/day): {total_energy:.2f} MJ/day")

non_pelletized_weight = total_production * 2 #kg/day
non_pelletized_energy = total_distance * non_pelletized_weight * truck_fuel_efficiency #MJ/day
print(f"Total energy for trucking (non-pelletized) (MJ/day): {non_pelletized_energy:.2f} MJ/day")


Total energy for trucking (MJ/day): 6283740.56 MJ/day
Total energy for trucking (non-pelletized) (MJ/day): 7289139.05 MJ/day


### PITTSBURGH

Case 1 - 150,000 Barrels/day 

42003 - Allegheny County,
42007 - Beaver County, 
42019 - Butler County, 
42125 - Washington County,
42129 - Westmoreland County,
39029 - Columbiana County,
42073 - Lawrence County,
54029 - Hancock County,
42005 - Armstrong County

Case 2 - 10,000 Barrels/day 

42003 - Allegheny County,


In [15]:
# Reference county: Allegheny County, PA (Pittsburgh)
pittsburgh = (40.468, -79.981)   # (lat, lon)

# Case 1 counties (150k bbl/day)
case1 = {
    "42003 - Allegheny County, PA": (40.468, -79.981),
    "42007 - Beaver County, PA": (40.685, -80.349),
    "42019 - Butler County, PA": (40.911, -79.916),
    "42125 - Washington County, PA": (40.195, -80.252),
    "42129 - Westmoreland County, PA": (40.311, -79.467),
    "39029 - Columbiana County, OH": (40.769, -80.77),
    "42073 - Lawrence County, PA": (40.992, -80.334),
    "54029 - Hancock County, WV": (40.528, -80.566),
    "42005 - Armstrong County, PA": (40.815, -79.464),
}

def compute(case_name, counties):
    print(f"\n=== {case_name} ===")
    total = 0.0
    for name, coords in counties.items():
        d = haversine(pittsburgh, coords)  # defaults to km
        total += d
        print(f"{name}: {d:.2f} km")
    print(f"--> Sum of distances: {total:.2f} km")
    return total 

total = compute("Case 1 (150k bbl/day)", case1)



=== Case 1 (150k bbl/day) ===
42003 - Allegheny County, PA: 0.00 km
42007 - Beaver County, PA: 39.35 km
42019 - Butler County, PA: 49.56 km
42125 - Washington County, PA: 38.07 km
42129 - Westmoreland County, PA: 46.90 km
39029 - Columbiana County, OH: 74.53 km
42073 - Lawrence County, PA: 65.42 km
54029 - Hancock County, WV: 49.91 km
42005 - Armstrong County, PA: 58.24 km
--> Sum of distances: 421.98 km


Now, total production (dry kg/day), as calculated in Task A -

Total Production gathered: 8715272.078709677 kg ~ 8715.272 tonnes

In [16]:
total_distance = total*1.3 #kms
total_production = 8715.272 #tonnes/day (calculated in task A for Kansas City)
pelletized_weight = total_production * (100/58) #kg/day

total_energy = total_distance * pelletized_weight * truck_fuel_efficiency #MJ/day
print(f"Total energy for trucking (MJ/day): {total_energy:.2f} MJ/day")

non_pelletized_weight = total_production * 2 #kg/day
non_pelletized_energy = total_distance * non_pelletized_weight * truck_fuel_efficiency #MJ/day
print(f"Total energy for trucking (non-pelletized) (MJ/day): {non_pelletized_energy:.2f} MJ/day")

Total energy for trucking (MJ/day): 12364650.01 MJ/day
Total energy for trucking (non-pelletized) (MJ/day): 14342994.02 MJ/day
